<a href="https://colab.research.google.com/github/rebeca07-pedrozo/DANE/blob/main/DANE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install openpyxl beautifulsoup4 --quiet

In [7]:
import os, time, re, hashlib
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import warnings

In [8]:
warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [9]:
download_folder = Path("/content/Descargas/DANE")
download_folder.mkdir(parents=True, exist_ok=True)

In [10]:
for file in download_folder.glob("*"):
    file.unlink()


In [11]:
from google.colab import files
uploaded = files.upload()

Saving Fuentes de información - Colombia (1).xlsx to Fuentes de información - Colombia (1).xlsx


In [13]:
df_urls = pd.read_excel("Fuentes de información - Colombia (1).xlsx", sheet_name="DANE")
print(df_urls.head())


  Categoría                                      Variable  \
0  Economía                            Empleo y desempleo   
1  Economía  Indicador de Seguimiento a la Economía (ISE)   
2  Economía         Índice de Precios al Consumidor (IPC)   
3  Economía         Índice de Precios del Productor (IPP)   
4  Economía         Índice de Producción Industrial (IPI)   

                                                 URL  
0  https://www.dane.gov.co/index.php/estadisticas...  
1  https://www.dane.gov.co/index.php/estadisticas...  
2  https://www.dane.gov.co/index.php/estadisticas...  
3  https://www.dane.gov.co/index.php/estadisticas...  
4  https://www.dane.gov.co/index.php/estadisticas...  


In [14]:
all_data_dane = []
for idx, row in df_urls.iterrows():
    url = row['URL']
    category = row['Categoría']
    variable = row['Variable']
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Error accediendo a la URL: {url}")
            continue
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a')
        for link in links:
            href = link.get('href', '')
            if re.search(r"\.xlsx?$", href, re.IGNORECASE):
                from urllib.parse import urljoin
                if href.startswith("/"):
                    href = urljoin(url, href)
                title = link.get('title', '').strip()
                if title:
                    all_data_dane.append({
                        "Categoría": category,
                        "Variable": variable,
                        "Título": title,
                        "Enlace": href
                    })
    except Exception as e:
        print(f"Error procesando la URL {url}: {e}")

df_data = pd.DataFrame(all_data_dane)
df_data = df_data[df_data['Título'] != '']
df_data['Nombre_Archivo'] = df_data['Enlace'].apply(lambda x: os.path.basename(x).split('?')[0])
print(df_data.head())

print("Archivos a descargar:", len(df_data))
for idx, row in df_data.iterrows():
    file_url = row['Enlace']
    file_name = row['Nombre_Archivo']
    if not re.search(r"\.xlsx?$", file_url, re.IGNORECASE):
        continue
    download_path = download_folder / file_name
    try:
        with requests.get(file_url, stream=True) as r:
            r.raise_for_status()
            with open(download_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"Archivo descargado exitosamente: {file_name}")
    except Exception as e:
        print(f"Error al descargar {file_name}: {e}")
print("Descarga de archivos completada.")

Error procesando la URL https://geoportal.dane.gov.co/servicios/descarga-y-metadatos/datos-geoestadisticos/?cod=112: HTTPSConnectionPool(host='geoportal.dane.gov.co', port=443): Max retries exceeded with url: /servicios/descarga-y-metadatos/datos-geoestadisticos/?cod=112 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7a13bd6cced0>, 'Connection to geoportal.dane.gov.co timed out. (connect timeout=None)'))
  Categoría            Variable  \
0  Economía  Empleo y desempleo   
1  Economía  Empleo y desempleo   
2  Economía  Empleo y desempleo   
3  Economía  Empleo y desempleo   
4  Economía  Empleo y desempleo   

                                              Título  \
0         Anexo Mercado laboral febrero 2025 en xlsx   
1      Anexo desestacionalizado febrero 2025 en xlsx   
2                 Anexo actualización GEIH CNPV 2018   
3              Ver Indicadores de empleo 1958 - 2018   
4  Indicadores de empleo por categoría ocupaciona...   

            

In [15]:
import os
for file in os.listdir("/content/Descargas/DANE"):
    print(file)


PIB-VLR-AGREG-DEPTO-CORRIENTES-1992.xls
PIB-VLR-AGREG-DEPTO-CONSTANTES-1991.xls
anex-PIBDep-Regiones-2023p.xlsx
PIB-VLR-AGREG-DEPTO-CORRIENTES-2000.xls
DCD-area-proypoblacion-dep-2005-2019.xlsx
anex-GastoConstantes-IVtrim2024.xlsx
PIBTOTALPERCACONSTANTES-base1975.xls
anexo-MESEP-nacional-2020.xls
anex-IPC-grupos-ingresos-media-mar2025.xlsx
DCD-areaproypoblacion-dep-1993-2004.xlsx
PIB-VLR-AGREG-DEPTO-CONSTANTES-1992.xls
DCD-area-proypoblacion-Mun-2005-2019.xlsx
PIB-VLR-AGREG-DEPTO-CORRIENTES-1991.xls
Anexo_RELAB_naranja_ene_19_mar_22.xlsx
PIB-VLR-AGREG-DEPTO-CORRIENTES-2001.xls
PIB-VLR-AGREG-DEPTO-CORRIENTES-1998.xls
PIB-VLR-AGREG-DEPTO-CONSTANTES-2004.xls
DCD-area-proypoblacion-Mun-1985-1994.xlsx
Anexo_RELAB_naranja_ene_19_dic_21.xlsx
anex-PM-SerieEmpalmadaIndicadores2012-2023.xlsx
anex-IPP-VariacionesProcedencia-mar2025.xlsx
anex-IPC-grupos-ingresos-pobres-mar2025.xlsx
Pibtotalresumencorrientesyconstantes-base1975.xls
anex-PM-DeflactorLineaspobrezaExtrema-2023.xlsx
DCD-area-sexo-edad-

In [18]:
import os

folder_path = "/content/Descargas/DANE"
total_archivos = len(os.listdir(folder_path))

print(f"Total de archivos descargados: {total_archivos}")



Total de archivos descargados: 160
